In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

In [ ]:
img_rows,img_cols = 28,28
num_classes=10
num_epochs=30
batch_size = 64
learning_rate = 0.001

In [ ]:
def to_categorical(y,num_classes):
    return np.eye(num_classes,dtype = 'int64')[y]

In [ ]:
def data_prep(raw):
    out_y = to_categorical(raw.label,num_classes)
    num_imgs = raw.shape[0]
    x_array = raw.values[:,1:]
    x_shaped_array = x_array.reshape(num_imgs,1,img_rows,img_cols)
    out_x = x_shaped_array/255
    return out_x,out_y

In [ ]:
train_file = '../input/digit-recognizer/train.csv'
train_data = pd.read_csv(train_file)
x_train,y_train = data_prep(train_data)
#print(x.shape)
x_train=torch.Tensor(x_train)
y_train=torch.Tensor(y_train)

In [ ]:
import torch.utils.data as data_utils
train_set = data_utils.TensorDataset(x_train,y_train)

In [ ]:
n_total = len(train_data)
n_train = int(0.8*(n_total))
n_val = int(0.2*(n_total))

In [ ]:
import torch.utils.data as data_utils
train_set = data_utils.TensorDataset(x_train,y_train)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
train_loader =data_utils.DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)

In [ ]:
from torchvision.models.resnet import ResNet, BasicBlock
class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, 
            kernel_size=(7, 7), 
            stride=(2, 2), 
            padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(
            super(MnistResNet, self).forward(x), dim=-1)

In [ ]:
model_ft = MnistResNet()

model = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.006,cycle_momentum=False)

In [ ]:
total_steps = len(train_loader)
loss_list = []
acc_list = []
lr_list = []

for epoch in range(num_epochs):
    for i,(inputs,labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        output = model(inputs)
        loss = criterion(output,torch.max(labels,1)[1])
        loss_list.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        total = labels.size(0)
        _,predicted = torch.max(output.data,1)
        labels = torch.max(labels,1)[1]

        correct = (predicted == labels).sum().item()
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                          (correct / total) * 100))


In [ ]:
def test_prep(raw):
    num_imgs = raw.shape[0]
    x_array = raw.values[:,:]
    x_shaped_array = x_array.reshape(num_imgs,1,img_rows,img_cols)
    out_x = x_shaped_array/255
    return out_x

In [ ]:
model.eval()
test_file = '../input/digit-recognizer/test.csv'
test_data = pd.read_csv(test_file)
x_test = test_prep(test_data)
x_test = torch.Tensor(x_test)
x_test = x_test.to(device)
y_pred = model(x_test)
pred = torch.max(y_pred,1)[1]
print(pred)

In [ ]:
print(len(pred))

In [ ]:
results = pd.Series(pred.cpu(),name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

In [ ]:
submission.to_csv("cnn_mnist_datagen.csv",index=False)